In [116]:
import os, sys, glob, shutil, json
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import cv2

from PIL import Image
import numpy as np

from tqdm import tqdm, tqdm_notebook

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

### Task2 图像数据读取及扩增（预处理）

pytorch输入数据PipeLine一般遵循一个“三步走”的策略，一般pytorch 的数据加载到模型的操作顺序是这样的：

① 创建一个 Dataset 对象。必须实现__len__()、__getitem__()这两个方法，这里面会用到transform对数据集进行扩充。

② 创建一个 DataLoader 对象。它是对DataSet对象进行迭代的，一般不需要实现。

③ 循环遍历这个 DataLoader 对象。将img, label加载到模型中进行训练。

————————————————

版权声明：本文为CSDN博主「LoveMIss-Y」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/qq_27825451/article/details/96130126

## 步骤1：定义好读取图像的Dataset

In [ ]:
#重载数据读取逻辑

class SVHNDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):#读取路径列表
        self.img_path = img_path
        self.img_label = img_label 
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None

    def __getitem__(self, index):#按条目index处理
        img = Image.open(self.img_path[index]).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)
        
        # 设置最长的字符长度为5个
        lbl = np.array(self.img_label[index], dtype=np.int)
        lbl = list(lbl)  + (5 - len(lbl)) * [10]# 补齐标注输出定长5列表，列表元素取值0-10。
        return img, torch.from_numpy(np.array(lbl[:5]))# 截取定长5的数据

    def __len__(self):#返回数据条目数
        return len(self.img_path)

## 步骤2：定义好训练数据和验证数据的Dataset

In [ ]:
#读取图片路径并按图片名称排序
train_path = glob.glob('input/mchar_train/*.png')
train_path.sort()
#读取json文件中的标注，list格式[0-9]
train_json = json.load(open('input/mchar_train.json'))
train_label = [train_json[x]['label'] for x in train_json]
print(len(train_path), len(train_label))

#训练样本处理
train_loader = torch.utils.data.DataLoader(
    SVHNDataset(train_path, train_label,
                transforms.Compose([                      #transforms打包
                    #训练数据的数据扩增=======================
                    transforms.Resize((64, 128)),         # 图片缩放到64*128
                    transforms.RandomCrop((60, 120)),     # 随机剪裁至60*120
                    transforms.ColorJitter(0.3, 0.3, 0.2),# 随机修改亮度、对比度和饱和度
                                                              # 当为a时，从[max(0, 1 - a), 1 + a]中随机选择。
                                                              # 当为（a，b）时，从[a, b]中选择。
                    transforms.RandomRotation(5),         # 加⼊入随机旋转，（+-degree）
                    #转换为训练格式==========================
                    transforms.ToTensor(),                # 将图⽚片转换为pytorch 的tesntor
                                                              # 应用了torchvision.transforms.ToTensor，其作用是
                                                              #（ Converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255]
                                                              #   to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0] ）    
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                                          # Normalize(mean, std)，对图像像素进⾏标准化，减mean除std
                                                              # Using the mean and std of Imagenet is a common practice. They are calculated based on millions of images. 
                                        
    ])), 
    batch_size=40, # 每批样本个数
    shuffle=True,  # 是否打乱顺序
    #num_workers=10,# 读取的线程个数，CPU为GPU预读取
)
##################################################
##################################################
##################################################
##################################################

val_path = glob.glob('input/mchar_val/*.png')
val_path.sort()
val_json = json.load(open('input/mchar_val.json'))
val_label = [val_json[x]['label'] for x in val_json]
print(len(val_path), len(val_label))

val_loader = torch.utils.data.DataLoader(
    SVHNDataset(val_path, val_label,
                transforms.Compose([
                    transforms.Resize((60, 120)),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=40, 
    shuffle=False, 
    #num_workers=10,
)

In [117]:
models.resnet50()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [118]:
models.resnet34()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## 步骤3：定义好字符分类模型，使用renset18的模型作为特征提取模块

In [ ]:
class SVHN_Model1(nn.Module):
    def __init__(self):#初始化，列举使用到的层
        super(SVHN_Model1, self).__init__()
                
        model_conv = models.resnet18(pretrained=True)#复用resnet结构及参数
        #resnet18中默认(avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
        model_conv.avgpool = nn.AdaptiveAvgPool2d(1) 
        '''二维自适应均值池化，输出效果（例子torch.Size([10,3,1,1])），resnet18相同。此处添加为了兼容各种模型，例如AlexNet (6*6)'''
        #model_conv中，(avgpool): AdaptiveAvgPool2d(output_size=1)
        model_conv = nn.Sequential(*list(model_conv.children())[:-1])
        # 建立Sequential顺序模型，以model_conv模型结构（去掉最后一层fc(512,1000)）
        self.cnn = model_conv
        
        #定义5个全联接层。1分5产生5个数字输出
        self.fc1 = nn.Linear(512, 11)
        self.fc2 = nn.Linear(512, 11)
        self.fc3 = nn.Linear(512, 11)
        self.fc4 = nn.Linear(512, 11)
        self.fc5 = nn.Linear(512, 11)
    
    def forward(self, img):#前向传播结构，CNN+5*全联接层用于输出结果
        feat = self.cnn(img)#先做卷积
        # print(feat.shape)
        feat = feat.view(feat.shape[0], -1)#将卷积的输出拉伸为一行
        '''input首先经过卷积层，此时的输出x是包含batchsize维度为4的tensor，即(batchsize，channels，x，y)，x.size(0)指batchsize的值。x = x.view(x.size(0), -1)简化x = x.view(batchsize, -1)。
view()函数的功能根reshape类似，用来转换size大小。x = x.view(batchsize, -1)中batchsize指转换后有几行，而-1指在不告诉函数有多少列的情况下，根据原tensor数据和batchsize自动分配列数。
————————————————
版权声明：本文为CSDN博主「whut_ldz」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/whut_ldz/article/details/78882532
'''
        c1 = self.fc1(feat)
        c2 = self.fc2(feat)
        c3 = self.fc3(feat)
        c4 = self.fc4(feat)
        c5 = self.fc5(feat)
        return c1, c2, c3, c4, c5 #输出结果：5个分类器

In [ ]:
model_conv = models.resnet18(pretrained=True)
#print('0',model_conv)
model_conv.avgpool = nn.AdaptiveAvgPool2d(1)#二元自适应均值池化，输出大小1*1
#print('1',model_conv)
model_conv = nn.Sequential(*list(model_conv.children())[:-1])# Sequential顺序模型；除了最后一个取全部
#print('2',model_conv)

# Task4 模型训练与验证

## 步骤4：定义好训练、验证和预测模块

- 训练集（Train Set）用于训练及调整模型参数，tag已知；
- 验证集（Validation Set）用于验证模型经度及调整模型超参数，tag已知；
- 测试集（Test Set） 验证模型的泛化能力，tag未知。

组织与【测试集】分布相同的【验证集】，验证并调整由【训练集】训练得到的模型。因此，要保证训练集-验证集-测试集的类别分布是一致的。

本次比赛给定了验证集，直接使用。

如未给出验证集，需要从训练集中抽取一部分作为验证集：

1、 留出法（Hold-Out），数据较大时使用，直接划出一部分

2、 交叉验证法（CV），数据较少时，数据多分块，并重复使用留出法

3、 自助采样法（BootStrap），数据较少时，有放回采样。


In [ ]:
def train(train_loader, model, criterion, optimizer):#minibatch训练一个epoch
    # 切换模型为训练模式
    model.train()
    '''
        model.train() ：启用 BatchNormalization 和 Dropout
        model.eval()  ：不用 BatchNormalization 和 Dropout
    '''
    train_loss = []
    
    for i, (input, target) in enumerate(train_loader):

        if use_cuda:
            input = input.cuda()
            target = target.cuda()
            
        c0, c1, c2, c3, c4 = model(input)
        target = target.long()
        '''
        b = torch.rand(n,n)#得到floattensor，
        b = b.long()       #得到longtensor，用于标注分类结果,criterion函数计算要求。
        '''
        loss = criterion(c0, target[:, 0])*2 + \
                criterion(c1, target[:, 1])*1.5 + \
                criterion(c2, target[:, 2])*1.2 + \
                criterion(c3, target[:, 3]) + \
                criterion(c4, target[:, 4])
        
        loss /= 6#计算一个batch的loss值

        #反向传播更新一次
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 100 == 0:#每100batch输出一次
            print(loss.item())
        
        train_loss.append(loss.item())
    return np.mean(train_loss)

def validate(val_loader, model, criterion):
    # 切换模型为预测模型
    model.eval()
    val_loss = []

    # 不记录模型梯度信息
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            if use_cuda:
                input = input.cuda()
                target = target.cuda()
            
            c0, c1, c2, c3, c4 = model(input)
            target = target.long()
            loss = criterion(c0, target[:, 0]) + \
                    criterion(c1, target[:, 1]) + \
                    criterion(c2, target[:, 2]) + \
                    criterion(c3, target[:, 3]) + \
                    criterion(c4, target[:, 4])
            # loss /= 6
            val_loss.append(loss.item())
    return np.mean(val_loss)

def predict(test_loader, model, tta=10):#TTA（Test Time Augmentation）测试时增强
    model.eval()
    test_pred_tta = None
    
    # TTA 次数
    for _ in range(tta):
        test_pred = []
    
        with torch.no_grad():
            for i, (input, target) in enumerate(test_loader):
                if use_cuda:
                    input = input.cuda()
                
                c0, c1, c2, c3, c4 = model(input)
                output = np.concatenate([
                    c0.data.numpy(), 
                    c1.data.numpy(),
                    c2.data.numpy(), 
                    c3.data.numpy(),
                    c4.data.numpy()], axis=1)

                test_pred.append(output)
        
        test_pred = np.vstack(test_pred)#将test_pred中相同形状的数组按垂直方向叠加
        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred
    
    return test_pred_tta

## 步骤5：迭代训练和验证模型


In [ ]:
#criterion = nn.CrossEntropyLoss()

'''损失函数criterion，交叉商CrossEntropyLoss
    Pytorch中CrossEntropyLoss()函数的主要是将softmax-log-NLLLoss合并到一块得到的结果。
    1、Softmax后的数值都在0~1之间，所以ln之后值域是负无穷到0。
    2、然后将Softmax之后的结果取log，将乘法改成加法减少计算量，同时保障函数的单调性 。
    3、NLLLoss的结果就是把上面的输出与Label对应的那几个值拿出来，求均值并取反。
'''
#optimizer = torch.optim.Adam(model.parameters(), 0.001)
'''优化器optimizer：自适应矩估计 Adam（Adaptive moment estimation）
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

params (iterable) – 待优化参数的iterable或者是定义了参数组的dict
lr (float, 可选) – 学习率（默认：1e-3）
betas (Tuple[float, float], 可选) – 用于计算梯度以及梯度平方的运行平均值的系数（默认：0.9，0.999）
eps (float, 可选) – 为了增加数值计算的稳定性而加到分母里的项（默认：1e-8）
weight_decay (float, 可选) – 权重衰减（L2惩罚）（默认: 0）
————————————————
版权声明：本文为CSDN博主「Ibelievesunshine」的原创文章，遵循CC 4.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
原文链接：https://blog.csdn.net/Ibelievesunshine/article/details/99624645
'''

In [ ]:
model = SVHN_Model1()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 0.001)
best_loss = 1000.0

use_cuda = False #GPU驱动

if use_cuda:
    model = model.cuda()

for epoch in range(5):
    if epoch ==10:
        optimizer = torch.optim.Adam(model.parameters(), 0.001/10)
        
    train_loss = train(train_loader, model, criterion, optimizer)
    val_loss = validate(val_loader, model, criterion)
    
    val_label = [''.join(map(str, x)) for x in val_loader.dataset.img_label]
    val_predict_label = predict(val_loader, model, 1)
    
    val_predict_label = np.vstack([#预测结果
        val_predict_label[:, :11].argmax(1),#取最大值位置，即预测值
        val_predict_label[:, 11:22].argmax(1),
        val_predict_label[:, 22:33].argmax(1),
        val_predict_label[:, 33:44].argmax(1),
        val_predict_label[:, 44:55].argmax(1),
    ]).T

    val_label_pred = []
    for x in val_predict_label:
        val_label_pred.append(''.join(map(str, x[x!=10])))#将[1,2,3,4,5]->'12345'
    
    val_char_acc = np.mean(  np.array(val_label_pred) == np.array(val_label)  )
    
    print('Epoch: {0}, Train loss: {1} \t Val loss: {2}'.format(epoch, train_loss, val_loss))
    print(val_char_acc)
    # 记录下验证集精度
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), './model.pt')

## 步骤6：对测试集样本进行预测，生成提交文件

In [ ]:
test_path = glob.glob('input/mchar_test_a/*.png')
test_path.sort()
test_label = [[1]] * len(test_path)#格式化dataset用来占位
print(len(val_path), len(val_label))

test_loader = torch.utils.data.DataLoader(
    SVHNDataset(test_path, test_label,
                transforms.Compose([
                    transforms.Resize((64, 128)),
                    transforms.RandomCrop((60, 120)),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=40, 
    shuffle=False, 
    num_workers=10,
)


model = model.load_state_dict(torch.load('../working/model.pt'))#读取测试集上“最优”模型参数
test_predict_label = predict(test_loader, model, 1)

#test_label = [''.join(map(str, x)) for x in test_loader.dataset.img_label]
test_predict_label = np.vstack([
    test_predict_label[:, :11].argmax(1),
    test_predict_label[:, 11:22].argmax(1),
    test_predict_label[:, 22:33].argmax(1),
    test_predict_label[:, 33:44].argmax(1),
    test_predict_label[:, 44:55].argmax(1),
]).T

test_label_pred = []
for x in test_predict_label:
    test_label_pred.append(''.join(map(str, x[x!=10])))
    
import pandas as pd
df_submit = pd.read_csv('input/mchar_sample_submit_A.csv')
df_submit['file_code'] = test_label_pred
df_submit.to_csv('renset18.csv', index=None)

# Task 5 模型集成

## 进展

0531-迁移到Kaggle GPU环境进行训练，epoch增加到20，测试了resnet18/resnet50，结果差不多，0.62左右；

0602-学习安晟直播，调整代码。每10个epoch学习率缩减到1/10；做predict时torch.load读取最好一次的参数（deadline中的小bug？）

## 学习理念及技巧

### 观察数据

- 问题背景，数据格式

- 数据分布

- 数据质量（脏数据、重复数据等）

### 搭建初始框架

- 固定随机种子，方便重现问题。

- Don't be a hero! 确保每一步的正确性。

- 将输入进网络的数据进行可视化，Tensorboard等。

- 设置合理指标，掌控训练过程。

### baseline基础优化

- 初始学习率选择，1e-2或1e-3

- 优化器选择，Adam

- 学习率阶段性下降策略

- 使用预训练模型

### 调参原则

- 厘清当前主要矛盾确定调参方向

- 调参过程中单一变量原则

### CV提高方法

- 研究并复现经典算法架构

- 先学会库调用，通过设置断点debug研究代码